# Modèle prescriptif de la détermination des prix - version compacte

Ce notebook est la version compacte du modèle d'optimisation. Contrairement à la version détaillée dans laquelle nous avons ajouté explicitement chaque équation, nous allons maintenant automatiser le processus de génération du modèle en utilisant des fonctionnalités de pyomo. Nous ne nous attendons pas à ce que vous soyez en mesure d'écrire tout ce code. Toutefois, vous devriez comprendre ce que fait chaque bloc. 

Plus particulièrement, nous souhaitons créer un **modèle compact** similaire au modèle de détermination des prix pour le cas Rue La La.

![modèle_compact](https://raw.githubusercontent.com/acedesci/scanalytics/master/FR/S09_Retail_Analytics_2/_static/modele_compact.png?raw=true)

*Rappelez-vous toutefois que $k$ signifie ici le prix moyen et non pas la somme des prix comme dans le modèle de Rue La La.*

In [1]:
import pandas as pd

import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    if "google.colab" in sys.modules:
        !pip install -q pyomo
    else:
        !conda install -c conda-forge pyomo
    assert(shutil.which("pyomo"))
    
if not (shutil.which("glpsol") or os.path.isfile("glpsol")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge glpk 
        except:
            pass
    assert(shutil.which("glpsol") or os.path.isfile("glpsol"))

import pyomo.environ as pe

# Bloc 1: données d'entrée
Nous avons préparé les entrées de données dans deux fichiers, c'est-à-dire,
1. **'predictedSales_Prob1.csv'**. C'est un problème à petite échelle. Il est identique au problème de la démonstration avec le modèle détaillé.
2. **'predictedSales_Prob2.csv'**. C'est un problème à grande échelle. Celui-ci contient un nombre beaucoup plus élevé de variables et de contraintes pour refléter le contexte réel.

Veuillez vous concentrer principalement sur le fichier *'predictedSales_Prob1.csv'* car vous pourrez voir le même modèle que dans la deuxième démonstration. Vous pouvez également essayer *'predictedSales_Prob2.csv'* si vous souhaitez voir le modèle à grande échelle.

In [2]:
# Le paramètre SIZE doit être défini à 'small' ou 'large'
SIZE = 'small'

if SIZE == 'small':
    # petit exemple
    url = 'https://raw.githubusercontent.com/acedesci/scanalytics/master/data/predictedSales_Prob1.csv'
elif SIZE == 'large':
    # grand exemple
    url = 'https://raw.githubusercontent.com/acedesci/scanalytics/master/data/predictedSales_Prob2.csv'
else:
    raise ValueError("SIZE doit être défini à 'small' ou 'large'")

predDemand = pd.read_csv(url)
print(predDemand.shape)
predDemand.head()

(12, 9)


,AVG_PRICE_VALUE,UPC,PRICE,PRICE_p2,FEATURE,DISPLAY,TPR_ONLY,RELPRICE,PRED_SALES
0,3.0,1600027528,2.5,6.25,0,0,0,0.833333,94.9
1,3.0,1600027528,3.0,9.00,0,0,0,1.000000,67.0
2,3.0,1600027528,3.5,12.25,0,0,0,1.166667,46.4
3,3.0,1600027564,2.5,6.25,0,0,0,0.833333,24.1
4,3.0,1600027564,3.0,9.00,0,0,0,1.000000,22.6


Avec le nouveau jeu de données, nous devons d'abord vérifier le nombre de valeurs de prix moyen, car nous devons exécuter le modèle d'optimisation pour chaque valeur du prix moyen.

In [3]:
productList = predDemand['UPC'].unique()
priceList = predDemand['PRICE'].unique()
avgPriceList = predDemand['AVG_PRICE_VALUE'].unique()
inputColumns = ['AVG_PRICE_VALUE', 'UPC', 'PRICE', 'PRED_SALES']

print(f"Liste des produits (i): {productList}")
print(f"Liste des prix (j): {priceList}")
print(f"Liste des prix moyens (k): {avgPriceList}")

Liste des produits (i): [1600027528 1600027564 3000006340 3800031829]
Liste des prix (j): [2.5 3.  3.5]
Liste des prix moyens (k): [3.]


# Bloc 2: préparation des paramètres d'entrée pour le modèle
Nous pouvons choisir la valeur de $k$ que nous voulons utiliser dans le modèle d'optimisation à partir de la colonne `'AVG_PRICE_VALUE'` des données. Dans *'predictedSales_Prob1.csv'*, il n'y a qu'un seul choix de prix moyen (soit 3,0\\$), tandis qu'il y a 3 prix moyens différents dans *'predictedSales_Prob2.csv'*.

Si vous souhaitez essayer différents choix de prix moyens, vous devez répéter cette procédure pour chaque valeur de prix moyen et enregistrer la solution optimale correspondante pour ensuite décider du prix moyen optimal et du prix optimal de chaque produit.

In [4]:
# Ici, nous choisissons la valeur de k (soit le prix moyen et non pas la somme des prix)
# Notez que k doit être parmi les choix où la prédiction a été faite
avgPriceValue =  avgPriceList[0] 

# Obtention des lignes correspondantes à k
predDemand_k = predDemand.loc[predDemand['AVG_PRICE_VALUE'] == avgPriceValue, inputColumns]
display(predDemand_k.head())

# Création d'un dictionnaire pour le modèle d'optimisation
D = {}
for price in priceList:  # j correspond ici directement au prix et non à un index
    for upc in productList:  # i correspond ici directement à un produit et non à un index
        # D_ijk avec k=avgPriceValue
        mask = (predDemand_k['UPC'] == upc) & (predDemand_k['PRICE'] == price)
        D[upc, price] = predDemand_k.loc[mask, 'PRED_SALES'].values[0]

print(f"D: {D}")

,AVG_PRICE_VALUE,UPC,PRICE,PRED_SALES
0,3.0,1600027528,2.5,94.9
1,3.0,1600027528,3.0,67.0
2,3.0,1600027528,3.5,46.4
3,3.0,1600027564,2.5,24.1
4,3.0,1600027564,3.0,22.6


D: {(1600027528, 2.5): 94.9, (1600027564, 2.5): 24.1, (3000006340, 2.5): 6.2, (3800031829, 2.5): 32.9, (1600027528, 3.0): 67.0, (1600027564, 3.0): 22.6, (3000006340, 3.0): 4.0, (3800031829, 3.0): 24.3, (1600027528, 3.5): 46.4, (1600027564, 3.5): 19.8, (3000006340, 3.5): 3.0, (3800031829, 3.5): 20.4}


# Bloc 3: création du modèle d'optimisation
## Bloc 3.1: déclarations des variables
Contrairement à la démonstration précédente, nous indexons les variables de décision et les paramètres directement par le produit et le prix plutôt que par leur indice. En effet, nous avons précédemment noté $x_{ij}=1$ si le prix associé à l'index $j$ est choisie pour le produit associé à l'index $i$, et 0 sinon. Maintenant, notre variable est notée $x_{1600027528,\ 3.0}=1$, ce qui signifie que le produit UPC '1600027528' sera vendu à 3,0\\$. La même remarque de notation s'applique à la demande prévue ($D_{ijk}$).

In [5]:
model = pe.ConcreteModel()

# Variables
model.x = pe.Var(productList, priceList, within=pe.Binary)

# Affichage du modèle
model.pprint()

1 Var Declarations
    x : Size=12, Index={1600027528, 3000006340, 1600027564, 3800031829}*{3.5, 2.5, 3.0}
        Key               : Lower : Value : Upper : Fixed : Stale : Domain
        (1600027528, 2.5) :     0 :  None :     1 : False :  True : Binary
        (1600027528, 3.0) :     0 :  None :     1 : False :  True : Binary
        (1600027528, 3.5) :     0 :  None :     1 : False :  True : Binary
        (1600027564, 2.5) :     0 :  None :     1 : False :  True : Binary
        (1600027564, 3.0) :     0 :  None :     1 : False :  True : Binary
        (1600027564, 3.5) :     0 :  None :     1 : False :  True : Binary
        (3000006340, 2.5) :     0 :  None :     1 : False :  True : Binary
        (3000006340, 3.0) :     0 :  None :     1 : False :  True : Binary
        (3000006340, 3.5) :     0 :  None :     1 : False :  True : Binary
        (3800031829, 2.5) :     0 :  None :     1 : False :  True : Binary
        (3800031829, 3.0) :     0 :  None :     1 : False :  True : 

## Bloc 3.2: ajout de la fonction objectif
Au lieu de saisir de manière itérative la valeur de chaque prix et les ventes prévues, nous pouvons simplement créer une boucle **pour** chaque produit et une boucle **pour** chaque prix. Le code ressemble maintenant beaucoup à l'équation générale $\sum_{i \in N} \sum_{j \in M} p_{j} \cdot \tilde{D}_{ijk} \cdot x_{ij}$ que nous avons vu précédemment.

Nous utilisons ici une expression pour définir la fonction objectif. Ce code est équivalent à celui donné en commentaire qui utilise une règle.

In [6]:
# fonction objectif
model.obj = pe.Objective(expr=sum(j * D[i,j] * model.x[i,j]
                                  for i in productList
                                  for j in priceList),
                         sense=pe.maximize)

# Code équivalent:
# def obj_rule(m):
#     # dans l'expression suivante, j correspond directement au prix
#     return sum(j * D[i,j] * m.x[i,j] for i in productList for j in priceList) 
# model.obj = pe.Objective(rule=obj_rule, sense=pe.maximize)

## Bloc 3.3: ajout des contraintes
### Contraintes 1: un choix de prix doit être choisi pour chaque produit

Il est possible d'ajouter plusieurs contraintes en utilisant une liste de contraintes.

Il est aussi possible d'ajouter plusieurs contraintes en même temps en utilisant des règles (voir le code commenté). Ces règles sont en fait des fonctions qui ont comme premier argument le modèle (ici, qu'on appelle le paramètre `m`) et comme arguments suivants les index nécessaires pour générer chacune des contraintes (ici, `i`). Cette contrainte est ensuite formée en spécifiant le ou les ensembles nécessaires, et la règle dans `pe.Constraint`.

In [7]:
# contraintes 1
model.priceChoiceUPC = pe.ConstraintList()
for i in productList:
    model.priceChoiceUPC.add(expr=sum(model.x[i,j] for j in priceList) == 1)

# Code équivalent:
# def priceChoiceUPC_rule(m, i):
#     return sum(m.x[i,j] for j in priceList) == 1 
# model.priceChoiceUPC = pe.Constraint(productList, rule=priceChoiceUPC_rule)  # productList fournie les valeurs de i

### Contrainte 2: la moyenne des prix de tous les styles concurrents doit être égal à $k$

On utilise ici une expression, mais il serait encore possible d'utiliser une règle. 

In [8]:
# contrainte 2
model.sumPrice = pe.Constraint(expr=sum(j * model.x[i,j] 
                                        for i in productList 
                                        for j in priceList) == 
                               avgPriceValue * len(productList))

# Code équivalent:
# def sumPrice_rule(m):
#     return sum(j * m.x[i,j] for i in productList for j in priceList) == avgPriceValue * len(productList) 
# model.sumPrice = pe.Constraint(rule=sumPrice_rule)

Nous pouvons maintenant afficher le modèle pour l'examiner avant de le résoudre.

In [9]:
model.pprint()

1 Var Declarations
    x : Size=12, Index={1600027528, 3000006340, 1600027564, 3800031829}*{3.5, 2.5, 3.0}
        Key               : Lower : Value : Upper : Fixed : Stale : Domain
        (1600027528, 2.5) :     0 :  None :     1 : False :  True : Binary
        (1600027528, 3.0) :     0 :  None :     1 : False :  True : Binary
        (1600027528, 3.5) :     0 :  None :     1 : False :  True : Binary
        (1600027564, 2.5) :     0 :  None :     1 : False :  True : Binary
        (1600027564, 3.0) :     0 :  None :     1 : False :  True : Binary
        (1600027564, 3.5) :     0 :  None :     1 : False :  True : Binary
        (3000006340, 2.5) :     0 :  None :     1 : False :  True : Binary
        (3000006340, 3.0) :     0 :  None :     1 : False :  True : Binary
        (3000006340, 3.5) :     0 :  None :     1 : False :  True : Binary
        (3800031829, 2.5) :     0 :  None :     1 : False :  True : Binary
        (3800031829, 3.0) :     0 :  None :     1 : False :  True : 

# Bloc 4: solution et interprétation
Enfin, nous appelons le solveur et obtenons la solution optimale. Pour le petit problème, nous pouvons voir que le produit '1600027528' est également vendu au prix de $2.5\$$, les produits '1600027564' et '3000006340' aussi au prix de $3.5\$$ et le produit '3800031829' toujours au prix de $2.5\$$. De plus, la valeur objective optimale est encore $399.3\$$.

Pour le grand problème, on obtient:
- $1750.50\$$ pour $k=2.5\$$,
- $1805.80\$$ pour $k=3.0\$$, et
- $1859.20\$$ pour $k=3.5\$$.

Donc, on suivrait la solution optimale de $k=3.5\$$. Vous pouvez exécuter le modèle du grand problème avec $k=3.5\$$ pour voir les valeurs de $x$.

In [10]:
# Résolution du modèle
pe.SolverFactory('glpk').solve(model)
model.display()

Model unknown

  Variables:
    x : Size=12, Index={1600027528, 3000006340, 1600027564, 3800031829}*{3.5, 2.5, 3.0}
        Key               : Lower : Value : Upper : Fixed : Stale : Domain
        (1600027528, 2.5) :     0 :   1.0 :     1 : False : False : Binary
        (1600027528, 3.0) :     0 :   0.0 :     1 : False : False : Binary
        (1600027528, 3.5) :     0 :   0.0 :     1 : False : False : Binary
        (1600027564, 2.5) :     0 :   0.0 :     1 : False : False : Binary
        (1600027564, 3.0) :     0 :   0.0 :     1 : False : False : Binary
        (1600027564, 3.5) :     0 :   1.0 :     1 : False : False : Binary
        (3000006340, 2.5) :     0 :   0.0 :     1 : False : False : Binary
        (3000006340, 3.0) :     0 :   0.0 :     1 : False : False : Binary
        (3000006340, 3.5) :     0 :   1.0 :     1 : False : False : Binary
        (3800031829, 2.5) :     0 :   1.0 :     1 : False : False : Binary
        (3800031829, 3.0) :     0 :   0.0 :     1 : False :

In [11]:
for i in productList:
    for j in priceList:
        if model.x[i,j].value == 1:
            print(i, j)

1600027528 2.5
1600027564 3.5
3000006340 3.5
3800031829 2.5
